# Aleph

The aim of this notebook is to use the Aleph system to perform ILP and learn a rule to identify buffer overflows. The rule should be of the form:

```
bug(AllocNode, WriteNode) :- predictateA, predicateB, ..., predicateZ.
```

In [1]:
import pandas as pd
import tempfile
import subprocess
import graph_visualisation as gv

## Data Preparation

We start by loading in our reduced set of buffer overflow examples, and their prolog representations. **If you're just reading, skip to the "Running Examples with Aleph" section.**

In [2]:
ilp_data = pd.read_csv("../data/ilp_dataset.csv.gz")
ilp_data = ilp_data.drop("Unnamed: 0", axis=1)
ilp_data.head()

,testcase_ID,filename,code,flaw,flaw_loc,bug,code_length
0,62804,000/062/804/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,33,False,1722
1,62821,000/062/821/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,35,False,1808
2,62852,000/062/852/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,30,False,1674
3,62853,000/062/853/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,33,False,2396
4,62854,000/062/854/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,33,False,2414


In [3]:
prolog = pd.read_csv("../data/ilp_prolog_data.csv.gz")
prolog = prolog.drop("Unnamed: 0", axis=1)
prolog.head()

,testcase_ID,flaw,bug,code_length,tree,source_map
0,-232086,CWE-122,True,1625,% START: Generated Prolog\n% NODE PROPERTIES \...,"% CODE\nsource_code(bad_232086_id_1_f_l_c_, ""p..."
1,-232012,CWE-122,True,1619,% START: Generated Prolog\n% NODE PROPERTIES \...,"% CODE\nsource_code(bad_232012_id_0_f_l_c_, ""p..."
2,-62917,CWE-121,True,1622,% START: Generated Prolog\n% NODE PROPERTIES \...,"% CODE\nsource_code(bad_62917_id_0_f_l_c_, ""p1..."
3,-62916,CWE-121,True,1649,% START: Generated Prolog\n% NODE PROPERTIES \...,"% CODE\nsource_code(bad_62916_id_1_f_l_c_, ""p1..."
4,-62915,CWE-121,True,1638,% START: Generated Prolog\n% NODE PROPERTIES \...,"% CODE\nsource_code(bad_62915_id_0_f_l_c_, ""p1..."


Next, we do a bunch of nasty stuff to find the particular nodes where the alloc and write nodes are in each prolog tree. 

In [4]:
juliet = pd.read_csv("../data/buffer_overflow_data.csv.gz")
juliet.drop("Unnamed: 0", axis=1)

prolog['source_code'] = ''
prolog['flaw_loc'] = 0
prolog['bad_code'] = ''

for i in range(len(prolog)):
    label = prolog.iloc[i].testcase_ID == juliet.testcase_ID
    prolog['source_code'].iloc[i] = juliet.loc[label].iloc[0].code
    prolog['flaw_loc'].iloc[i] = juliet.loc[label].iloc[0].flaw_loc
    
for i in range(len(prolog)):
    loc =  prolog.iloc[i].flaw_loc
    testcase_ID = prolog.iloc[i].testcase_ID
    if testcase_ID < 0:
        prolog['bad_code'].iloc[i] = prolog.iloc[i].source_code.split('\n')[loc-1].strip()[0:-1]
    else:
        prolog['bad_code'].iloc[i] = prolog.loc[prolog.testcase_ID == -testcase_ID].iloc[0].bad_code
        
good_examples = prolog[prolog['bug'] == False]
bad_examples = prolog[prolog['bug'] == True]

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
def extract_node_ids(tree):
    nodes = set()
    
    in_ast_section = False
    
    for line in tree.split('\n'):
        line = line.strip()
        
        if line == '% AST':
            in_ast_section = True
        
        if line == '% CFG':
            in_ast_section = False
            
        if line == '% REF':
            in_ast_section = False
        
        if in_ast_section and not line.startswith("%"):
            parent, child = line[4:-2].split(", ")
            nodes.add(parent)
            nodes.add(child)
            
    return nodes

In [6]:
# good nodes_g
nodes_g = set()

for i in range(len(good_examples)):
    good_nodes = extract_node_ids(good_examples.iloc[i].tree)
    nodes_g = set.union(nodes_g,good_nodes)

    
# bad nodes_b
nodes_b = set()

for i in range(len(bad_examples)):
    bad_nodes = extract_node_ids(bad_examples.iloc[i].tree)
    nodes_b = set.union(nodes_b,bad_nodes)

nodes = set.union(nodes_g,nodes_b)

In [7]:
def find_bug_id(good_examples, i = 0):
    tree = good_examples.source_map.iloc[i]
    parents = []
    children = []
    in_code_section = False
    temp = 0
    for line in tree.split('\n'):
        line = line.strip()
        
        if line == '% CODE':
            in_code_section = True
        
        if in_code_section and not line.startswith("%"):
            try:
                #import pdb; pdb.set_trace()
                parent, child = line[12:-3].split(', "')
                parents.append(parent)
                children.append(child)
            except:
                temp +=1
    zipbObj = zip(children, parents)
    bug_node = dict(zipbObj)
    
    loc = good_examples.iloc[i].flaw_loc
    source_code = good_examples.iloc[i].bad_code

    return bug_node[source_code]

In [8]:
bad_node = []
good_node = []
for i in range(20):
    bad_node.append(find_bug_id(bad_examples,i))
    good_node.append(find_bug_id(good_examples,i))

In [9]:
node_and_tree = []
for node in nodes:
    start, end = node.split('_id')
    node_and_tree.append('(' +start+ ', ' +node+ ')')

node_and_tree[0:5]

['(good_62913, good_62913_id_105_f_memmove_14_c_l_81_c_11_)',
 '(good_62854, good_62854_id_211_f_memcpy_03_c_l_24_c_0_)',
 '(bad_62867, bad_62867_id_32_f_l_c_)',
 '(good_62821, good_62821_id_126_f_loop_18_c_l_36_c_17_)',
 '(bad_62868, bad_62868_id_115_f_l_50_c_0_)']

In [10]:
node_tree_types = [
    'node_in_tree'+ node_tree +'.' for node_tree in node_and_tree
]

node_types = [
    'node('+ node_id +').' for node_id in nodes
]

In [11]:
def extract_tree_ids(tree):
    line = tree.split('\n')[2]
    line = line.strip()
    line = line.split('(')[1]
    tree_id = line.split('_id')[0]

    return tree_id

In [12]:
tree_ids = []
for tree in prolog.tree:
    tree_ids.append(extract_tree_ids(tree))

trees = [
    'tree('+ tree_label +').' for tree_label in tree_ids
]

Here we try to locate the buggy nodes. For each test case this will be:
  1. Location where the memory is _allocated_.
  2. Location where the memory is _written to_.
  
We already have code above to find the latter. Can we find the first as well?

In [13]:
bad_write_nodes = bad_node
good_write_nodes = good_node

In [14]:
# this code is horrible sorry. if you need to figure out what it does speak to Dan.

alloc_source_maps = [
    line
    for source_map in prolog['source_map']
    for line in source_map.split("\n")
    if 'data =' in line.lower() and 'alloc' in line.lower()
]
bad_alloc_nodes = [
    source_map.strip("source_code(").split(",")[0]
    for source_map in alloc_source_maps
    if 'bad' in source_map
]

good_alloc_nodes = [
    source_map.strip("source_code(").split(",")[0]
    for source_map in alloc_source_maps
    if 'good' in source_map
]

positive_examples = []
for write_node in bad_write_nodes:
    # find the alloc node which has a line number before the write
    write_node_id = write_node.strip("source_code(").split(",")[0] + "_"
    write_line = int(write_node_id.split("_")[-4])
    
    testcase_id = '_'.join(write_node_id.split("_")[0:2])
    
    potential_alloc_nodes = [node for node in bad_alloc_nodes if testcase_id in node]
    for alloc_node in potential_alloc_nodes:
        alloc_node_id = alloc_node.strip("source_code(").split(",")[0] + "_"
        alloc_line = int(alloc_node_id.split("_")[-4])
        
        if alloc_line <= write_line:
            positive_examples.append(
                "bug({},{}).".format(alloc_node_id, write_node_id)
            )
            
negative_examples = []
for write_node in good_write_nodes:
    # find the alloc node which has a line number before the write
    write_node_id = write_node.strip("source_code(").split(",")[0] + "_"
    write_line = int(write_node_id.split("_")[-4])
    
    testcase_id = '_'.join(write_node_id.split("_")[0:2])
    
    potential_alloc_nodes = [node for node in good_alloc_nodes if testcase_id in node]
    for alloc_node in potential_alloc_nodes:
        alloc_node_id = alloc_node.strip("source_code(").split(",")[0] + "_"
        alloc_line = int(alloc_node_id.split("_")[-4])
        
        if alloc_line <= write_line:
            negative_examples.append(
                "bug({},{}).".format(alloc_node_id, write_node_id)
            )

## Running Examples with Aleph

**Now for the interesting part.** Below is our template to create our scripts for aleph to use.

In [15]:
def filter_lines_by_testcase(string, testcases):
    return '\n'.join([
        line for line in string.split("\n")  if any(
            str(testcase) in line for testcase in testcases
        ) or not ("good_" in line or "bad_" in line)
    ])


def make_aleph_scripts(header, positive_examples, negative_examples, testcases=None):
    script_template = """
{header}

:- discontiguous ast/2.
:- discontiguous cfg/2.
:- discontiguous ancestor/2.
:- discontiguous runs_before/2.
:- discontiguous assignment/1.
:- discontiguous compMemberAccess/1.
:- discontiguous ref/2.
:- discontiguous sizeOf/1.
:- discontiguous alloc/1.
:- discontiguous writeToPointer/1.
:- discontiguous alloc_doesnt_check_sizeOf/1.
:- discontiguous pointer/1.
:- discontiguous sizeOfInt/1.
:- discontiguous array10/1.
:- discontiguous voidPointer/1.

%% Trees
{trees}

%% Types
{types}

%% Background knowledge
{bg_knowledge}


%% Rules for background knowledge
writeToPointer(A) :- ast(A,B), assignment(A), compMemberAccess(B). 

ancestor(A,C) :- ast(A,B), ancestor(B,C).
ancestor(A,C) :- ast(A,C).

runs_before(A,C) :- cfg(A,B), runs_before(B,C).
runs_before(A,C) :- cfg(A,C).

contains_sizeOf_call(A) :- ancestor(A, B), sizeof(B).
alloc_doesnt_check_sizeOf(A) :- alloc(A), not(contains_sizeOf_call(A)).

same_tree(A,B) :- node_in_tree(T,A), node_in_tree(T,B).
    """
    script_bg = script_template.format(
        header = header,
        trees =  '\n'.join(trees),
        types = '\n'.join(node_tree_types + node_types),
        bg_knowledge = '\n'.join(prolog['tree'])
    )

    script_template_pos = """

%% positive examples
{positive_examples}

    """
    
    script_template_neg = """
    
%% negative examples
{negative_examples}

    """
    script_pos = script_template_pos.format(
        positive_examples='\n '.join(positive_examples)
    )
    
    script_neg = script_template_neg.format(
        negative_examples='\n '.join(negative_examples)
    )
    
    # keep all testcases unless a testcases set is given in fn call
    if testcases:
        # remove all lines not related to this testcase
        script_bg = filter_lines_by_testcase(script_bg)
        script_pos = filter_lines_by_testcase(script_pos)
        script_neg = filter_lines_by_testcase(script_neg)
        
    return script_bg, script_pos, script_neg

Experimenting with aleph:

In [19]:
header = """

% body
:- mode(*,runs_before(+node,+node)).
:- mode(*,ancestor(+node,-node)).
:- mode(*,ancestor(-node,+node)).

:- mode(*, ref(+node,-node)).
:- mode(*, ref(-node,+node)).

:- mode(*, sizeOf(+node)).
:- mode(*, alloc(+node)).
:- mode(*, writeToPointer(+node)).

:- mode(*, alloc_doesnt_check_sizeOf(+node)).

:- mode(*, pointer(+node)).
:- mode(*, sizeOfInt(+node)).
:- mode(*, array10(+node)).
:- mode(*, voidPointer(+node)).

% determinations
:- determination(bug/2, ancestor/2).
:- determination(bug/2, runs_before/2).
:- determination(bug/2, ref/2).
:- determination(bug/2, sizeOf/1).
:- determination(bug/2, alloc/1).
:- determination(bug/2, writeToPointer/1).
:- determination(bug/2, alloc_doesnt_check_sizeOf/1).
:- determination(bug/2, pointer/1).
:- determination(bug/2, sizeOfInt/1).
:- determination(bug/2, array10/1).
:- determination(bug/2, voidPointer/1).

"""

script_bg, script_pos, script_neg = make_aleph_scripts(
    header,
    positive_examples,
    negative_examples,
)

runner_script = """
:- working_directory(_, "/tmp/aleph_scripts"),
[library(acuity)],
read_all(aleph_scripts),
set(interactive,true),
set(depth,30),
set(clauselength,20),
set(clauses,10),
set(minpos,2),
induce,
halt.
"""

!mkdir -p /tmp/aleph_scripts


with open("/tmp/runner.pl", "w") as f:
    f.write(runner_script)

with open("/tmp/aleph_scripts/aleph_scripts.b", "w") as f:
    f.write(script_bg)

with open("/tmp/aleph_scripts/aleph_scripts.f", "w") as f:
    f.write(script_pos)
    
with open("/tmp/aleph_scripts/aleph_scripts.n", "w") as f:
    f.write(script_neg)
    
!swipl -s /tmp/runner.pl > output.log